### Qwen3_RL_Deep_Research_Agent_training_with_unsolth(WIP)

### Installation

In [ ]:
# %%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm==0.8.2
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.2

In [ ]:
!nvidia-smi

In [ ]:
!pip install camel-ai -U

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
# %%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm==0.8.2
else:
    !pip install --no-deps unsloth vllm==0.8.2
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [ ]:
!pip install typing_extensions -U

### Unsloth

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

In [ ]:
!vllm serve IAAR-Shanghai/xVerify-0.5B-I  --tensor-parallel-size 1 --dtype=half

# import the xverify

In [ ]:
!git clone https://github.com/IAAR-Shanghai/xVerify.git && cd xVerify
%cd xVerify/
!pip install -r requirements.txt
!pip install vllm

In [ ]:
from src.xVerify.model import Model
from src.xVerify.eval import Evaluator

In [10]:
### xverify

import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
# def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
#     responses = [completion[0]['content'] for completion in completions]
#     q = prompts[0][-1]['content']
#     extracted_responses = [extract_xml_answer(r) for r in responses]
#     print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
#     return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]


# Reward functions

model_name = 'IAAR-Shanghai/xVerify-0.5B-I'  # Model name
url = 'http://localhost:8000/v1'  # Anonymized model path or URL
inference_mode = 'api'  # Inference mode, 'local' or 'api'
api_key = None  # API key used to access the model via API, if not available, set to None
model = Model(
    model_name=model_name,
    model_path_or_url=url,
    inference_mode=inference_mode,
    api_key=api_key
)
evaluator = Evaluator(model=model)


def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")

    rewards = []
    for r, a in zip(responses, answer):
        # evaluation
        result = evaluator.single_evaluate(
            question=q,
            llm_output=r,
            correct_answer=a
        )
        # result can be "Correct" or "Incorrect"

        # Assign reward values according to the evaluation results
        if result == "Correct":
            rewards.append(2.0)  # Reward is 2.0 when correct
        elif result == "Incorrect":
            rewards.append(0.0)  # Reward is 0.0 when incorrect
        else:
            raise ValueError(f"Unexpected evaluation result: {result}")

    return rewards


def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]


# # Reward function for reasoning traces
# def reasoning_trace_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
#     """
#     Evaluate the quality of reasoning traces and provide a reward score.
#     Scores are based on the depth, logical consistency, and relevance
#     of the reasoning to the final answer.
#     """
#     responses = [completion[0]["content"] for completion in completions]
#     rewards = []

#     for response in responses:
#         reward = 0.0
#         # 1. Check if the response contains reasoning tags
        # if "<reasoning>" in response and "</reasoning>" in response:
        #     # Extract the reasoning section
        #     reasoning = response.split("<reasoning>")[1].split("</reasoning>")[0].strip()
        #     # Extract the answer section (if exists)
        #     answer_text = ""
        #     if "<answer>" in response and "</answer>" in response:
        #         answer_text = response.split("<answer>")[1].split("</answer>")[0].strip()

        #     # 2. Assess the depth of reasoning (based on length and content)
        #     reasoning_lines = reasoning.split("\n")
        #     non_empty_lines = [line for line in reasoning_lines if line.strip()]

        #     # Base reward for including reasoning
        #     reward += 0.2

        #     # Additional reward for number of reasoning steps
        #     if len(non_empty_lines) >= 3:
        #         reward += 0.2
        #     if len(non_empty_lines) >= 5:
        #         reward += 0.1

        #     # 3. Evaluate the quality of reasoning

        #     # Check for mathematical calculations or logical steps
        #     has_calculations = any(["=" in line or "+" in line or "-" in line or "*" in line or "/" in line for line in non_empty_lines])
        #     if has_calculations:
        #         reward += 0.1

    #         # Check for intermediate results
    #         if len(non_empty_lines) > 1:
    #             # The last line is usually the conclusion, previous lines should contain intermediate steps
    #             has_intermediate_steps = any(["=" in line for line in non_empty_lines[:-1]])
    #             if has_intermediate_steps:
    #                 reward += 0.1

    #         # Check for logical connectors (therefore, thus, because, etc.)
    #         logic_connectors = ["therefore", "thus", "because", "first", "second", "finally", "hence"]
    #         has_logic_flow = any([any([connector in line.lower() for connector in logic_connectors]) for line in non_empty_lines])
    #         if has_logic_flow:
    #             reward += 0.1

    #         # 4. Check if the final answer aligns with the reasoning
    #         if answer_text and reasoning:
    #             # Simple check: whether the answer appears in the final part of the reasoning
    #             conclusion = " ".join(non_empty_lines[-2:]).lower()
    #             if answer_text.lower().strip() in conclusion:
    #                 reward += 0.2

    #     rewards.append(reward)

    # return rewards

In [ ]:
### original

import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

In [ ]:
# Single sample evaluation test
from src.xVerify.model import Model
from src.xVerify.eval import Evaluator

# initialization
model_name = 'xVerify-0.5B-I'  # Model name
url = 'https://your-anonymized-url/v1'  # Anonymized model path or URL
inference_mode = 'local'  # Inference mode, 'local' or 'api'
api_key = None  # API key used to access the model via API, if not available, set to None
model = Model(
    model_name=model_name,
    model_path_or_url=url,
    inference_mode=inference_mode,
    api_key=api_key
)
evaluator = Evaluator(model=model)

# input evaluation information,
question = "New steel giant includes Lackawanna site A major change is coming to the global steel industry and a galvanized mill in Lackawanna that formerly belonged to Bethlehem Steel Corp.\nClassify the topic of the above sentence as World, Sports, Business, or Sci/Tech."
llm_output = "The answer is Business."
correct_answer = "Business"

# evaluation
result = evaluator.single_evaluate(
    question=question,
    llm_output=llm_output,
    correct_answer=correct_answer
)
print(result)

In [ ]:
def english_tool_call_reward_func(prompts, completions, answer=None, **kwargs) -> list[float]:
    """
    Specialized reward function for tool calls in English scenarios.

    Focuses on evaluating the quality of toolkit calls with emphasis on browser
    interactions, search quality, and proper result handling.
    """
    responses = [completion[0]["content"] for completion in completions]
    questions = [prompt[-1]["content"] for prompt in prompts]
    rewards = []

    for i, (question, response) in enumerate(zip(questions, responses)):
        reward = 0.0

        try:
            # 1. Detect toolkit call patterns with focus on English formatting
            english_toolkit_patterns = [
                r"BrowserToolkit\(\)\.([a-zA-Z_]+)\(.*\)",
                r"browser_toolkit\.([a-zA-Z_]+)\(.*\)",
                r"web_toolkit\.([a-zA-Z_]+)\(.*\)",
                r"browser\.([a-zA-Z_]+)\(.*\)",
                r"SearchToolkit\(\)\.([a-zA-Z_]+)\(.*\)",
                r"search_toolkit\.([a-zA-Z_]+)\(.*\)",
            ]

            toolkit_calls = []
            for pattern in english_toolkit_patterns:
                matches = re.findall(pattern, response)
                toolkit_calls.extend(matches)

            # If toolkit calls found
            if toolkit_calls:
                # Base reward for using toolkits
                reward += 0.3

                # Additional reward for each unique toolkit function
                unique_calls = set(toolkit_calls)
                reward += min(0.1 * len(unique_calls), 0.4)

                # 2. Enhanced browser toolkit evaluation
                browser_patterns = [
                    "navigate", "click", "type", "scroll", "screenshot",
                    "get_text", "get_element", "wait_for", "browse"
                ]

                browser_calls = [call for call in toolkit_calls
                               if any(pattern in call.lower() for pattern in browser_patterns)]

                if browser_calls:
                    # Browser usage base reward
                    reward += 0.2

                    # Browser call sequence quality
                    # Check for logical sequence: navigate -> interaction -> extract
                    has_navigate = any("navigate" in call.lower() for call in browser_calls)
                    has_interaction = any(action in "".join(browser_calls).lower()
                                      for action in ["click", "type", "scroll"])
                    has_extraction = any(extract in "".join(browser_calls).lower()
                                    for extract in ["get_text", "get_element", "screenshot"])

                    # Reward complete browser interaction flow
                    if has_navigate and has_interaction and has_extraction:
                        reward += 0.5
                    elif has_navigate and (has_interaction or has_extraction):
                        reward += 0.3
                    elif has_navigate:
                        reward += 0.1

                    # Check for URL quality in navigation
                    url_pattern = r"(https?://[^\s\"\']+)"
                    urls = re.findall(url_pattern, response)
                    if urls:
                        # Valid URL with domain-specific targeting
                        if any(domain in urls[0].lower() for domain in
                              ["amazon", "google", "wikipedia", "github", "stackoverflow"]):
                            reward += 0.2

                # 3. Search toolkit quality evaluation
                search_patterns = ["search", "query", "find", "lookup"]
                search_calls = [call for call in toolkit_calls
                              if any(pattern in call.lower() for pattern in search_patterns)]

                if search_calls:
                    # Search usage base reward
                    reward += 0.1

                    # Check for search query quality
                    query_pattern = r"query\s*=\s*[\"\'](.*?)[\"\']"
                    queries = re.findall(query_pattern, response)

                    if queries:
                        # Reward based on query specificity and length
                        avg_query_length = sum(len(q) for q in queries) / len(queries)
                        reward += min(avg_query_length / 40, 0.3)

                        # Check if queries are relevant to the question
                        question_words = set(re.findall(r'\b[a-zA-Z]{3,}\b', question.lower()))
                        query_matches = sum(1 for q in queries
                                         for word in question_words
                                         if word in q.lower())

                        reward += min(0.05 * query_matches, 0.3)

                # 4. Result processing evaluation for toolkit calls
                result_processing_patterns = [
                    r"([a-zA-Z_][a-zA-Z0-9_]*)\s*=\s*.*toolkit\.",  # Variable assignment
                    r"print\(.*toolkit\.",  # Direct output
                    r"if\s+.*toolkit\.",    # Conditional processing
                    r"for\s+.*toolkit\.",   # Iteration over results
                    r"analyze_results\(",   # Analysis function
                    r"process_data\("       # Processing function
                ]

                for pattern in result_processing_patterns:
                    if re.search(pattern, response):
                        reward += 0.2
                        break

                # 5. Error handling specifically for toolkit operations
                toolkit_error_handling = [
                    r"try\s*:.*toolkit.*except",
                    r"if\s+.*not\s+.*toolkit",
                    r"if\s+.*error.*toolkit",
                    r"if\s+.*failed.*toolkit"
                ]

                for pattern in toolkit_error_handling:
                    if re.search(pattern, response, re.DOTALL):
                        reward += 0.2
                        break

                # 6. Multi-toolkit integration
                # Check if multiple different toolkit types are used together
                has_browser = any("browser" in call.lower() for call in toolkit_calls)
                has_search = any("search" in call.lower() for call in toolkit_calls)
                has_function = any("function" in call.lower() for call in toolkit_calls)

                toolkit_count = sum([has_browser, has_search, has_function])
                if toolkit_count > 1:
                    reward += 0.3 * (toolkit_count - 1)

                # 7. Bonus for advanced browser operations
                advanced_browser_ops = [
                    "wait_for_selector", "evaluate_javascript", "get_cookies",
                    "handle_dialog", "multiple_tabs", "fill_form"
                ]

                for op in advanced_browser_ops:
                    if op.lower() in response.lower():
                        reward += 0.15
                        break

            # Limit maximum reward
            reward = min(reward, 2.5)  # Higher cap for browser-focused rewards

        except Exception as e:
            print(f"Error in English toolkit evaluation: {e}")
            reward = 0.1

        rewards.append(reward)

    return rewards

In [ ]:
import re
from camel.agents import ChatAgent
from camel.configs import ChatGPTConfig
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

# Load OpenAI model
model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4_1,
    model_config_dict=ChatGPTConfig().as_dict(),
)

# Create ChatAgent
camel_agent = ChatAgent(model=model)


def generate_verifier_and_reward_function(question: str, response: str, correct_answer: str) -> dict:
    """
    Use an OpenAI large language model to generate code for a verifier and reward function.

    Args:
        question (str): The content of the question.
        response (str): The answer generated by the model.
        correct_answer (str): The correct answer.

    Returns:
        dict: A dictionary containing the generated verifier and reward function code.
    """
    # Build prompt message
    prompt = f"""
    You are an AI code generator. Your task is to analyze the following:

    Question: {question}
    Response: {response}
    Correct Answer: {correct_answer}

    Based on the above information, please generate:
    1. A Python function called `verify_response` that takes `response` and `correct_answer` as inputs and returns a boolean indicating whether the response is correct.
    2. A Python function called `calculate_reward` that takes `response`, `correct_answer`, and any other necessary inputs, and returns a float reward value.

    Make sure the generated code is self-contained and executable.

    Example output format:
    {{
        "verifier_code": "def verify_response(response, correct_answer): ...",
        "reward_code": "def calculate_reward(response, correct_answer): ..."
    }}
    """

    # Call the OpenAI model to generate code
    response_info = camel_agent.step(prompt)
    generated_code = eval(response_info.msgs[0].content)

    return generated_code


def dynamic_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """
    Dynamically generate a reward function and compute reward values.

    Args:
        prompts (list): List of input questions.
        completions (list): List of answers generated by the model.
        answer (list): List of correct answers.

    Returns:
        list[float]: Reward value for each answer.
    """
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]

    rewards = []

    for r, a in zip(responses, answer):
        # Dynamically generate verifier and reward function code
        code = generate_verifier_and_reward_function(q, r, a)

        # Define a secure execution environment
        verifier_code = code.get("verifier_code", "")
        reward_code = code.get("reward_code", "")

        try:
            # Dynamically load verifier code
            exec(verifier_code, globals())
            # Dynamically load reward function code
            exec(reward_code, globals())

            # Call the generated verifier and reward function
            is_correct = verify_response(r, a)  # type: ignore
            reward = calculate_reward(r, a)  # type: ignore

            # If the verifier judges it correct, keep the reward; otherwise set to zero
            if not is_correct:
                reward = 0.0

            rewards.append(reward)
        except Exception as e:
            print(f"Error executing generated code: {e}")
            rewards.append(0.0)  # Default reward is 0 if execution fails

    return rewards


# Example usage
if __name__ == "__main__":
    # Example data
    prompts = [[{"role": "system", "content": SYSTEM_PROMPT}, {"role": "user", "content": "What is 2 + 2?"}]]
    completions = [[{"content": "<reasoning>\nThe sum of 2 and 2 is 4.\n</reasoning>\n<answer>\n4\n</answer>"}]]
    answer = ["4"]

    # Calculate reward values
    rewards = dynamic_reward_func(prompts, completions, answer)
    print("Rewards:", rewards)

In [ ]:
ef specialized_browser_tool_call_reward_func(prompts, completions, answer=None, **kwargs) -> list[float]:
    """
    Highly specialized reward function focusing exclusively on browser toolkit usage.

    Evaluates sophisticated browser interactions including navigation chains,
    data extraction quality, form interactions, and web scraping patterns.
    """
    responses = [completion[0]["content"] for completion in completions]
    questions = [prompt[-1]["content"] for prompt in prompts]
    rewards = []

    for i, (question, response) in enumerate(zip(questions, responses)):
        reward = 0.0

        try:
            # Skip evaluation if no browser toolkit usage detected
            if not any(pattern in response for pattern in
                     ["BrowserToolkit", "browser_toolkit", "web_toolkit", "navigate", "browser."]):
                rewards.append(0.0)
                continue

            # 1. Comprehensive browser interaction flow assessment
            # Extract all browser-related function calls in sequence
            browser_call_sequence = re.findall(
                r"(browser|BrowserToolkit|web_toolkit)\s*\.\s*([a-zA-Z_]+)\s*\(([^)]*)\)",
                response
            )

            if browser_call_sequence:
                # Base reward for using browser toolkit
                reward += 0.4

                # Create ordered list of browser operations
                browser_ops = [call[1] for call in browser_call_sequence]

                # Categorize operations
                navigation_ops = ["navigate", "goto", "open", "visit"]
                interaction_ops = ["click", "type", "fill", "select", "check", "scroll", "hover"]
                extraction_ops = ["get_text", "get_element", "screenshot", "get_attribute", "get_html"]
                waiting_ops = ["wait_for", "wait_until", "sleep", "delay", "timeout"]

                # Check for ideal operation sequences
                has_navigation = any(op in "".join(browser_ops) for op in navigation_ops)
                has_waiting = any(op in "".join(browser_ops) for op in waiting_ops)
                has_interaction = any(op in "".join(browser_ops) for op in interaction_ops)
                has_extraction = any(op in "".join(browser_ops) for op in extraction_ops)

                # 2. Evaluate operation sequence quality
                # Ideal flow: navigate -> wait -> interact -> extract
                operation_sequence_score = 0.0

                # Check if operations occur in a logical order
                if has_navigation:
                    operation_sequence_score += 0.2

                    # Check if wait operations follow navigation
                    if has_waiting and any(w in op for op in browser_ops for w in waiting_ops) and browser_ops.index(next((op for op in browser_ops if any(w in op for w in waiting_ops)), "")) > browser_ops.index(next((op for op in browser_ops if any(n in op for n in navigation_ops)), "")):
                        operation_sequence_score += 0.2

                    # Check if interactions follow navigation/waiting
                    if has_interaction:
                        operation_sequence_score += 0.2

                        # Check if extraction follows interaction
                        if has_extraction and any(e in op for op in browser_ops for e in extraction_ops) and browser_ops.index(next((op for op in browser_ops if any(e in op for e in extraction_ops)), "")) > browser_ops.index(next((op for op in browser_ops if any(i in op for i in interaction_ops)), "")):
                            operation_sequence_score += 0.3

                reward += operation_sequence_score

                # 3. Parameter quality assessment
                parameter_quality_score = 0.0

                # Check for specific selectors in operations
                for call in browser_call_sequence:
                    params = call[2]

                    # CSS selector specificity
                    if re.search(r"[\"\']\s*(\.|#)[a-zA-Z0-9_-]+\s*[\"\']\s*", params):
                        parameter_quality_score += 0.1

                    # XPath usage
                    if re.search(r"[\"\']//[^\"\']+(\\[@[^\"\'])|/[^\"\']+[\"\']", params):
                        parameter_quality_score += 0.15

                    # URL quality for navigation
                    if call[1] in navigation_ops and re.search(r"[\"\'](https?://[^\"\']+)[\"\']", params):
                        parameter_quality_score += 0.1

                    # Text content specification
                    if "text" in params or "content" in params:
                        parameter_quality_score += 0.1

                reward += min(parameter_quality_score, 0.5)

                # 4. Advanced browser usage patterns
                advanced_usage_score = 0.0

                # Form interaction pattern (multiple field inputs followed by submission)
                form_pattern = re.search(r"type\s*\([^)]+\)[^}]+type\s*\([^)]+\)[^}]+(click|submit)", response, re.DOTALL)
                if form_pattern:
                    advanced_usage_score += 0.3

                # Login sequence detection
                login_pattern = re.search(r"(username|email|login|password)[^}]+(type|fill)[^}]+(submit|click|login)", response, re.DOTALL | re.IGNORECASE)
                if login_pattern:
                    advanced_usage_score += 0.2

                # Multiple page navigation sequence
                multi_page_navigation = len([op for op in browser_ops if any(n in op for n in navigation_ops)]) > 1
                if multi_page_navigation:
                    advanced_usage_score += 0.2

                # Data extraction and processing
                data_extraction_processing = re.search(r"get_[a-z_]+\([^)]*\)[^}]+for\s+[^}]+in", response, re.DOTALL)
                if data_extraction_processing:
                    advanced_usage_score += 0.3

                reward += min(advanced_usage_score, 0.8)

                # 5. Error handling and robustness
                robustness_score = 0.0

                # Try-except blocks around browser operations
                try_except_pattern = re.search(r"try\s*:[^}]+(browser|BrowserToolkit|web_toolkit)[^}]+except", response, re.DOTALL)
                if try_except_pattern:
                    robustness_score += 0.2

                # Conditional checks before/after operations
                conditional_checks = re.search(r"if\s+[^}]+(browser|BrowserToolkit|web_toolkit)", response, re.DOTALL)
                if conditional_checks:
                    robustness_score += 0.2

                # Waiting for elements before interaction
                if has_waiting and has_interaction:
                    wait_before_interact = False
                    for i in range(len(browser_ops)-1):
                        if any(w in browser_ops[i] for w in waiting_ops) and any(inter in browser_ops[i+1] for inter in interaction_ops):
                            wait_before_interact = True
                            break

                    if wait_before_interact:
                        robustness_score += 0.2

                reward += min(robustness_score, 0.5)

            # Limit maximum reward - higher cap for specialized browser function
            reward = min(reward, 3.0)

        except Exception as e:
            print(f"Error in specialized browser evaluation: {e}")
            reward = 0.1

        rewards.append(reward)

    return rewards


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [16]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [15]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

AttributeError: 'Model' object has no attribute 'config'

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on unsloth [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.



In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )